## Load building attributes data

In [2]:
# %pip install pyarrow
# %pip install --upgrade pandas
# %pip install xlsxwriter


import pyarrow
import pandas as pd
import numpy as np
from pathlib import Path


import sys, platform
print("=== Environment Info ===")
print(f"Python version : {sys.version.split()[0]}")
print(f"Python env     : {sys.executable}")
print(f"Platform       : {platform.platform()}")
print(f"numpy          : {np.__version__}")
print(f"pandas         : {pd.__version__}")
print("========================")


=== Environment Info ===
Python version : 3.11.13
Python env     : c:\Users\pc\.conda\envs\geo_env\python.exe
Platform       : Windows-10-10.0.22631-SP0
numpy          : 1.26.4
pandas         : 2.3.2


In [3]:

# Define the file path
folder = Path(r"E:\London\colouringbritain\data-extract-2025-09-01")
csv_file = folder / "building_attributes.csv"

# Subfolder for cached files
cache_dir = folder / "cache"
cache_dir.mkdir(exist_ok=True)   # create if not exists



In [ ]:


# # Load CSV --- 1. very slow and cause errors
# df_building = pd.read_csv(
#     csv_file, 
#     engine="pyarrow",
#     dtype_backend="pyarrow", # arrow dtypes (lower memory usage)
# )

# # Print first 5 rows
# print(df_building.head())


# 2. 2nd way to load csv data 
import pyarrow.csv as pv
import pyarrow.parquet as pq
# 1) Read with Arrow (tolerates newlines-in-values)
table = pv.read_csv(
    csv_file,
    read_options=pv.ReadOptions(block_size=1<<26),  # 64MB blocks (tweak if needed)
    parse_options=pv.ParseOptions(newlines_in_values=True)
)

# 2) Save once to Parquet for instant reloads later
parquet_path = cache_dir / "building_attributes.parquet"
pq.write_table(table, parquet_path, compression="snappy")

# 3) (optional) Convert to pandas right now
df_building = table.to_pandas(types_mapper=pd.ArrowDtype)
print(df_building.head())


Save as Pickle format for fast reload

✅ Pros: very fast, preserves dtypes exactly.
⚠️ Cons: Python-specific, not portable across languages.

In [ ]:


# # Pickle file path
# pkl_file = cache_dir / "building_attributes.pkl"

# # Load CSV (if pickle not already saved)
# if not pkl_file.exists():
#     print("Pickle not found, reading CSV and saving as pickle...")
#     df_building = pd.read_csv(csv_file)
#     df_building.to_pickle(pkl_file)
# else:
#     print("Pickle found, loading directly...")
#     df_building = pd.read_pickle(pkl_file)

# print(df_building.head())




# # Fast reload
# parquet_path = cache_dir / "building_attributes.parquet"
# df_building = pd.read_parquet(parquet_path, engine="pyarrow")


### Data summary

In [10]:
print(df_building.columns.tolist())



# for col in df_building.columns:
#     print(f"\n--- {col} ---")
#     try:
#         # value_counts is efficient and sorts by frequency
#         vals = df_building[col].value_counts(dropna=False).head(10)
#         print(vals)
#     except Exception as e:
#         print(f"Could not process column {col}: {e}")

# # Basic info: rows, columns, dtypes, memory
# print("\n=== DataFrame Info ===")
# print(df_building.info())

# # Summary stats for numeric columns
# print("\n=== Summary Statistics (numeric) ===")
# print(df_building.describe().T)

# # Summary stats for categorical/text columns
# print("\n=== Summary (categorical) ===")
# print(df_building.describe(include=['object', 'category']).T)

# # Count missing values per column
# print("\n=== Missing Values ===")
# print(df_building.isna().sum())


['building_id', 'ref_toid', 'ref_osm_id', 'revision_id', 'location_name', 'location_name_link', 'location_number', 'location_street', 'location_line_two', 'location_town', 'location_postcode', 'location_address_source', 'location_address_links', 'location_latitude', 'location_longitude', 'current_landuse_group', 'current_landuse_order', 'building_attachment_form', 'date_change_building_use', 'date_year', 'date_lower', 'date_upper', 'date_source', 'date_source_detail', 'date_link', 'facade_year', 'facade_upper', 'facade_lower', 'facade_source', 'facade_source_detail', 'size_storeys_attic', 'size_storeys_core', 'size_storeys_basement', 'size_storeys_source_type', 'size_storeys_source_links', 'size_height_apex', 'size_height_apex_source_type', 'size_height_apex_source_links', 'size_height_eaves', 'size_height_eaves_source_type', 'size_height_eaves_source_links', 'size_floor_area_ground', 'size_floor_area_total', 'size_floor_area_source_type', 'size_floor_area_source_links', 'size_width_fr

### data summary - columns 

In [9]:
import pandas as pd

# Path for output
out_xlsx = cache_dir / "building_attributes_summary.xlsx"

# Create a dict to hold results
summary = {}

for col in df_building.columns:
    try:
        vals = df_building[col].value_counts(dropna=False).head(10)
        # convert to DataFrame for nice export
        summary[col] = vals.reset_index().rename(columns={"index": col, col: "count"})
    except Exception as e:
        summary[col] = pd.DataFrame({"error": [str(e)]})

# Export to Excel (each column in a separate sheet)
with pd.ExcelWriter(out_xlsx, engine="xlsxwriter") as writer:
    for col, df in summary.items():
        # Excel sheet names must be ≤31 chars
        sheet_name = str(col)[:31]
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Summary exported to {out_xlsx}")


Summary exported to E:\London\colouringbritain\data-extract-2025-09-01\cache\building_attributes_summary.xlsx


### select columns

In [ ]:
cols_selected = [
    'building_id', 'ref_toid', 'ref_osm_id',      
    #  'revision_id', 
    'location_name', 
    #  'location_name_link', 'location_number', 'location_street', 'location_line_two', 
    'location_town', 'location_postcode', 
    #  'location_address_source', 'location_address_links', 
    'location_latitude', 'location_longitude', 
    'current_landuse_group', 'current_landuse_order', 'building_attachment_form', 
    'date_change_building_use', 'date_year', 
    #  'date_lower', 'date_upper', 'date_source', 'date_source_detail', 'date_link', 
    #  'facade_year', 'facade_upper', 'facade_lower', 'facade_source', 'facade_source_detail', 
    'size_storeys_attic', 'size_storeys_core', 'size_storeys_basement', 
    #  'size_storeys_source_type', 'size_storeys_source_links', 
    'size_height_apex', 
    #  'size_height_apex_source_type', 'size_height_apex_source_links', 'size_height_eaves', 'size_height_eaves_source_type', 'size_height_eaves_source_links', 
    'size_floor_area_ground', 'size_floor_area_total', 
    #  'size_floor_area_source_type', 'size_floor_area_source_links', 'size_width_frontage', 
    'sust_breeam_rating', 'sust_dec', 'sust_retrofit_date', 'construction_core_material', 'construction_secondary_materials', 'construction_roof_covering', 
    #  'planning_portal_link', 'planning_in_conservation_area_id', 'planning_in_conservation_area_url', 'planning_conservation_area_name', 'planning_in_list', 
    #  'planning_list_id', 'planning_list_cat', 'planning_list_grade', 'planning_heritage_at_risk_url', 'planning_world_list_id', 'planning_glher_url', 
    #  'planning_in_apa_url', 'planning_local_list_url', 'planning_historic_area_assessment_url', 
    'is_domestic', 
    #  'is_domestic_source', 'is_domestic_links', 
    #  'community_type_worth_keeping_total', 'likes_total', 'survival_status', 'survival_source', 'survival_source_links', 'disaster_type', 'disaster_severity', 
    #  'disaster_assessment_method', 'disaster_source_link', 'disaster_start_date', 'disaster_end_date', 'size_far_ratio', 'size_far_ratio_source_type', 
    #  'size_far_ratio_source_links', 'size_plot_area_total', 'size_plot_area_total_source_type', 'size_plot_area_total_source_links', 'size_parcel_geometry', 
    #  'size_parcel_geometry_source_type', 'size_parcel_geometry_source_links', 
    'context_front_garden', 'context_back_garden', 'context_flats_garden', 
    #  'context_garden_source_type', 'context_garden_source_type', 'context_street_width', 'context_street_width_source_type', 'context_street_width_source_links', 
    #  'context_pavement_width', 'context_pavement_width_source_type', 'context_pavement_width_source_links', 'context_street_geometry', 
    #  'context_street_geometry_source_type', 'context_street_geometry_source_links', 
    'context_green_space_distance', 
    #  'context_green_space_distance_source_links', 
    'context_tree_distance', 
    # 'context_tree_distance_source_type', 'context_tree_distance_source_links', 
    # 'age_cladding_date', 'age_cladding_date_source_type', 'age_cladding_date_source_links', 'age_extension_date', 'age_extension_date_source_type', 
    # 'age_extension_date_source_links', 
    'age_retrofit_date', 
    #  'age_retrofit_date_source_type', 'age_retrofit_date_source_links'
    ]


df_building_subset = df_building[cols_selected]

print(df_building_subset.head())



### save subset data

In [ ]:
## 1. Save as CSV
# out_csv =  cache_dir / "building_attributes_subset.csv"
# df_building_subset.to_csv(out_csv, index=False, encoding="utf-8")
# print(f"Subset saved to: {out_csv}")

# import pyarrow.csv as pv
# import pyarrow.parquet as pq
# out_csv =  cache_dir / "building_attributes_subset.csv"
# df_building_subset = pv.read_csv(
#     out_csv,
#     read_options=pv.ReadOptions(block_size=1<<26),  # 64MB blocks (tweak if needed)
#     parse_options=pv.ParseOptions(newlines_in_values=True)
# )



## 2. Save as Parquet (fast reload for analysis)
# out_parquet = cache_dir / "building_attributes_subset.parquet"
# df_building_subset.to_parquet(out_parquet, engine="pyarrow", compression="snappy", index=False)
# print(f"Saved Parquet: {out_parquet}")



## 3. Save as Feather (optional, even faster reload if only in Python)
# import pyarrow.feather as feather
# out_feather = cache_dir / "building_attributes_subset.feather"
# feather.write_feather(df_building_subset, out_feather)
# print(f"Saved Feather: {out_feather}")

Saved Feather: E:\London\colouringbritain\data-extract-2025-09-01\cache\building_attributes_subset.feather


### reload

In [15]:
import pyarrow.feather as feather

out_feather = cache_dir / "building_attributes_subset.feather"
df_building_subset = feather.read_feather(out_feather)


## Load building footprint data

In [ ]:
# import geopandas as gpd

# # Path to your shapefile (make sure to use raw string r"" or double slashes \\ for Windows paths)
# shapefile_path = r"E:\London\colouringbritain\London building footprints_2025-09-11\buildings.shp"

# # Load into GeoDataFrame
# gdf = gpd.read_file(shapefile_path)

# print(gdf.head())
# print(gdf.crs)  # coordinate reference system


   fid          area  perimeter layer  \
0  1.0  3.503437e-09   0.000266  TEST   
1  2.0  3.830134e-09   0.000308  TEST   
2  3.0  4.776945e-09   0.000326  TEST   
3  4.0  5.876390e-08   0.002627  TEST   
4  5.0  2.593630e-09   0.000232  TEST   

                                                path  \
0  /home/mateusz/Desktop/vm_share/not_upload_tool...   
1  /home/mateusz/Desktop/vm_share/not_upload_tool...   
2  /home/mateusz/Desktop/vm_share/not_upload_tool...   
3  /home/mateusz/Desktop/vm_share/not_upload_tool...   
4  /home/mateusz/Desktop/vm_share/not_upload_tool...   

                                            geometry  
0  POLYGON ((530850.030 200467.340, 530853.140 20...  
1  POLYGON ((530946.580 200101.170, 530942.780 20...  
2  POLYGON ((530958.680 200117.570, 530962.290 20...  
3  POLYGON ((526208.730 198238.610, 526212.740 19...  
4  POLYGON ((524271.910 197156.219, 524277.781 19...  
EPSG:27700


### subset data

In [ ]:
# # Drop specific columns
# buildings_gdf = gdf.drop(columns=["layer", "path"])


# # Save cleaned GeoDataFrame to a new file (GeoPackage format) 
# out_gpkg = cache_dir / "buildings_clean.gpkg"
# buildings_gdf.to_file(out_gpkg, driver="GPKG")


# # Save cleaned GeoDataFrame to Parquet format -- the fastest option if you’ll only use the data in Python.
# out_parquet = cache_dir / "buildings_clean.parquet"
# buildings_gdf.to_parquet(out_parquet, index=False)



# # Save as Feather format -- very fast reload in Python
# out_feather = cache_dir / "buildings_clean.feather"
# buildings_gdf.to_feather(out_feather)


### reload

In [ ]:
import geopandas as gpd

buildings_parquet = cache_dir / "buildings_clean.parquet"

# Reload instantly
buildings_gdf = gpd.read_parquet(buildings_parquet)


# OR
# # Reload (super quick)
# buildings_gdf = gpd.read_feather(out_feather)


## Merge building shp with attributes data

### reload building attributes

In [13]:
import pyarrow.feather as feather

out_feather = cache_dir / "building_attributes_subset.feather"
df_building_subset = feather.read_feather(out_feather)


df_building_subset.head()


,building_id,ref_toid,ref_osm_id,location_name,location_town,location_postcode,location_latitude,location_longitude,current_landuse_group,current_landuse_order,...,construction_secondary_materials,construction_roof_covering,is_domestic,context_front_garden,context_back_garden,context_flats_garden,context_green_space_distance,context_green_space_distance,context_tree_distance,age_retrofit_date
0,165387,osgb1000005167158,NaN,,,,51.57382,-0.15977,None,Residential,...,None,,,,,,NaN,NaN,NaN,NaN
1,1843,osgb1000001793684754,432890749.0,,,,51.46078,-0.30239,None,Residential,...,None,,,,,,NaN,NaN,NaN,NaN
2,1845,osgb1000003398193,833983224.0,,,,51.46143,-0.29687,None,Residential,...,None,,,,,,NaN,NaN,NaN,NaN
3,1807754,osgb5000005209570449,NaN,,,,51.39136,-0.13144,None,Residential,...,None,,,,,,NaN,NaN,NaN,NaN
4,1956,osgb1000003398334,NaN,,,,51.46031,-0.30045,None,Residential,...,None,,,,,,NaN,NaN,NaN,NaN


In [14]:
# Assuming your DataFrame is called df_building_subset
print("Unique values in current_landuse_group:")
print(df_building_subset["current_landuse_group"].dropna().unique())

print("\nUnique values in current_landuse_order:")
print(df_building_subset["current_landuse_order"].dropna().unique())


Unique values in current_landuse_group:
[]

Unique values in current_landuse_order:
['Residential' 'Community Services' 'Transport' '' 'Mixed Use'
 'Industry And Business' 'Retail' 'Recreation And Leisure'
 'Unclassified, presumed non-residential' 'Utilities And Infrastructure'
 'Vacant And Derelict' 'Defence' 'Agriculture And Fisheries' 'Minerals']


### reload building shp 

In [9]:
import geopandas as gpd

buildings_parquet = cache_dir / "buildings_clean.parquet"

# Reload instantly
buildings_gdf = gpd.read_parquet(buildings_parquet)


In [8]:

# Basic info: rows, columns, dtypes, memory
print("\n=== DataFrame Info ===")
print(buildings_gdf.info())
print(buildings_gdf.shape)

buildings_gdf.head()


=== DataFrame Info ===
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2223481 entries, 0 to 2223480
Data columns (total 4 columns):
 #   Column     Dtype   
---  ------     -----   
 0   fid        float64 
 1   area       float64 
 2   perimeter  float64 
 3   geometry   geometry
dtypes: float64(3), geometry(1)
memory usage: 67.9 MB
None
(2223481, 4)


,fid,area,perimeter,geometry
0,1.0,3.503437e-09,0.000266,"POLYGON ((530850.030 200467.340, 530853.140 20..."
1,2.0,3.830134e-09,0.000308,"POLYGON ((530946.580 200101.170, 530942.780 20..."
2,3.0,4.776945e-09,0.000326,"POLYGON ((530958.680 200117.570, 530962.290 20..."
3,4.0,5.876390e-08,0.002627,"POLYGON ((526208.730 198238.610, 526212.740 19..."
4,5.0,2.593630e-09,0.000232,"POLYGON ((524271.910 197156.219, 524277.781 19..."


### spatial join 

In [ ]:
import geopandas as gpd

# 1) Make points GeoDataFrame from lat/lon
df_pts = df_building_subset.dropna(subset=["location_latitude", "location_longitude"]).copy()

gdf_pts = gpd.GeoDataFrame(
    df_pts,
    geometry=gpd.points_from_xy(
        x=df_pts["location_longitude"],
        y=df_pts["location_latitude"],
        crs="EPSG:4326"   # WGS84
    ),
)

# 2) Match CRS to buildings_gdf (whatever it is)
if gdf_pts.crs != buildings_gdf.crs:
    gdf_pts = gdf_pts.to_crs(buildings_gdf.crs)

# 3) Spatial join: which building polygon each point falls within
#    (If buildings_gdf are polygons, 'within' is appropriate; use 'intersects' if you prefer)
pts_in_bldgs = gpd.sjoin(
    gdf_pts,
    buildings_gdf,
    how="left",
    predicate="within"
)

# Optional: Fallback nearest join for points not matched by within/intersects
# (useful if points are slightly outside the polygon boundaries)
if pts_in_bldgs["index_right"].isna().any():
    # Only run nearest for the unmatched rows to save time
    gdf_unmatched = gdf_pts.loc[pts_in_bldgs["index_right"].isna()]
    nearest = gpd.sjoin_nearest(
        gdf_unmatched,
        buildings_gdf,
        how="left",
        distance_col="nearest_dist"
    )
    # Fill the unmatched rows in the main result
    pts_in_bldgs.update(nearest)



# Save for fast reload later ------------------------------------------------------------------------------------------------
# Parquet (fastest for Python)
pts_in_bldgs.to_parquet(r"E:\London\colouringbritain\data-extract-2025-09-01\points_joined_buildings.parquet", index=False)

# GeoPackage (GIS-friendly)
pts_in_bldgs.to_file(r"E:\London\colouringbritain\data-extract-2025-09-01\points_joined_buildings.gpkg", driver="GPKG")

print(pts_in_bldgs.head())
